In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\programdata\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201103


In [5]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [25]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\定例\Web明細登録数")

input_new = "WEBMEISAI_CUST"
input_all = "Web明細登録数累積"

input_cust = "TOTAL_CUST"
input_cust_all = "有効口座数"

#out_put
output = "WEBMEISAI_CUST_ALL"
output2 = "TOTAL_CUST_ALL"


#output(result)
result = "Web明細Overview"
result_cha = "Web明細新規登録数_by_channel"
result_card = "Web明細新規登録数_by_card"
result_age = "Web明細新規登録数_by_age"

①Web明細登録データの読込＆集計

In [27]:
#過去累積分と更新分を合算し、CSV出力
df_exist = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
#文字化けするので修正
df_exist['age'] = df_exist['age'].str.replace('〜','-')

df_new = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')#文字化けするので修正
df_new['age'] = df_new['age'].str.replace('〜','-')

In [28]:
df_exist

,cp,year,month,card,channel,age,gender,new_account,no_of_account,free_f
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,0,21,NaN
1,1903,2019,3,ゴールド,オンライン,25-29歳,女性,1,2,NaN
2,1903,2019,3,ゴールド,オンライン,25-29歳,男性,0,23,NaN
3,1903,2019,3,ゴールド,オンライン,30-34歳,女性,0,18,NaN
4,1903,2019,3,ゴールド,オンライン,30-34歳,女性,1,2,NaN
...,...,...,...,...,...,...,...,...,...,...
6501,2008,2020,8,外商,郵送,75-79歳,男性,1,33,1.0
6502,2008,2020,8,外商,郵送,80歳以上,女性,0,133,1.0
6503,2008,2020,8,外商,郵送,80歳以上,女性,1,21,1.0
6504,2008,2020,8,外商,郵送,80歳以上,男性,0,183,1.0


In [29]:
df_new

,cp,year,month,card,channel,age,gender,new_account,free_f,no_of_account
0,2009,2020,9,ゴールド,オンライン,25-29歳,女性,0,0,16
1,2009,2020,9,ゴールド,オンライン,25-29歳,女性,1,0,1
2,2009,2020,9,ゴールド,オンライン,25-29歳,男性,0,0,22
3,2009,2020,9,ゴールド,オンライン,25-29歳,男性,1,0,1
4,2009,2020,9,ゴールド,オンライン,30-34歳,女性,0,0,39
...,...,...,...,...,...,...,...,...,...,...
379,2009,2020,9,外商,郵送,75-79歳,男性,1,1,128
380,2009,2020,9,外商,郵送,80歳以上,女性,0,1,154
381,2009,2020,9,外商,郵送,80歳以上,女性,1,1,53
382,2009,2020,9,外商,郵送,80歳以上,男性,0,1,230


In [30]:
df = pd.concat([df_exist,df_new])
df.to_csv(data_dir /f'{output}.csv', header=True ,index=None, encoding='shift-jis')

In [31]:
df

,cp,year,month,card,channel,age,gender,new_account,no_of_account,free_f
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,0,21,NaN
1,1903,2019,3,ゴールド,オンライン,25-29歳,女性,1,2,NaN
2,1903,2019,3,ゴールド,オンライン,25-29歳,男性,0,23,NaN
3,1903,2019,3,ゴールド,オンライン,30-34歳,女性,0,18,NaN
4,1903,2019,3,ゴールド,オンライン,30-34歳,女性,1,2,NaN
...,...,...,...,...,...,...,...,...,...,...
379,2009,2020,9,外商,郵送,75-79歳,男性,1,128,1.0
380,2009,2020,9,外商,郵送,80歳以上,女性,0,154,1.0
381,2009,2020,9,外商,郵送,80歳以上,女性,1,53,1.0
382,2009,2020,9,外商,郵送,80歳以上,男性,0,230,1.0


In [32]:
df = df[df['cp']>= 1803] #2018年3月以降に限定
df

,cp,year,month,card,channel,age,gender,new_account,no_of_account,free_f
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,0,21,NaN
1,1903,2019,3,ゴールド,オンライン,25-29歳,女性,1,2,NaN
2,1903,2019,3,ゴールド,オンライン,25-29歳,男性,0,23,NaN
3,1903,2019,3,ゴールド,オンライン,30-34歳,女性,0,18,NaN
4,1903,2019,3,ゴールド,オンライン,30-34歳,女性,1,2,NaN
...,...,...,...,...,...,...,...,...,...,...
379,2009,2020,9,外商,郵送,75-79歳,男性,1,128,1.0
380,2009,2020,9,外商,郵送,80歳以上,女性,0,154,1.0
381,2009,2020,9,外商,郵送,80歳以上,女性,1,53,1.0
382,2009,2020,9,外商,郵送,80歳以上,男性,0,230,1.0


In [33]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(6890, 10)
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            374, 375, 376, 377, 378, 379, 380, 381, 382, 383],
           dtype='int64', length=6890)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'new_account', 'no_of_account', 'free_f'],
      dtype='object')
cp                 int64
year               int64
month              int64
card              object
channel           object
age               object
gender            object
new_account        int64
no_of_account      int64
free_f           float64
dtype: object


In [34]:
#日付データの変換
df['cp'] = df['cp'] + 200000
df['cp_date'] = pd.to_datetime(df['cp'], format="%Y%m") + MonthEnd(1)
df.head()

,cp,year,month,card,channel,age,gender,new_account,no_of_account,free_f,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,0,21,NaN,2019-03-31
1,201903,2019,3,ゴールド,オンライン,25-29歳,女性,1,2,NaN,2019-03-31
2,201903,2019,3,ゴールド,オンライン,25-29歳,男性,0,23,NaN,2019-03-31
3,201903,2019,3,ゴールド,オンライン,30-34歳,女性,0,18,NaN,2019-03-31
4,201903,2019,3,ゴールド,オンライン,30-34歳,女性,1,2,NaN,2019-03-31


#日付データの変換
df['cp_date'] = (df['cp'] + 200000)*100 + 1
df['cp_date']=pd.to_datetime(df['cp_date'], format='%Y%m%d')
df.head()

In [35]:
#日付から時間を削除
df['cp_date'] = df['cp_date'].dt.strftime('%Y/%m/%d')
df.head()

,cp,year,month,card,channel,age,gender,new_account,no_of_account,free_f,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,0,21,NaN,2019/03/31
1,201903,2019,3,ゴールド,オンライン,25-29歳,女性,1,2,NaN,2019/03/31
2,201903,2019,3,ゴールド,オンライン,25-29歳,男性,0,23,NaN,2019/03/31
3,201903,2019,3,ゴールド,オンライン,30-34歳,女性,0,18,NaN,2019/03/31
4,201903,2019,3,ゴールド,オンライン,30-34歳,女性,1,2,NaN,2019/03/31


全体の入りと出を集計

In [36]:
q = """
        SELECT
        cp_date
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date
        """

df_web = sqldf(q, locals())
df_web.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2019/03/31,158235,5667,163902
1,2019/04/30,161036,6059,167095
2,2019/05/31,164514,7221,171735
3,2019/06/30,167583,7146,174729
4,2019/07/31,171889,8815,180704


In [37]:
#前月末を算出
q = """
        SELECT
        *
        ,LAG (eop_sum_acco, 1) OVER (ORDER BY cp_date) AS bop_sum_acco
        from 
            df_web
        group by
        cp_date
        """

df_web2 = sqldf(q, locals())
df_web2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco
0,2019/03/31,158235,5667,163902,NaN
1,2019/04/30,161036,6059,167095,163902.0
2,2019/05/31,164514,7221,171735,167095.0
3,2019/06/30,167583,7146,174729,171735.0
4,2019/07/31,171889,8815,180704,174729.0


In [38]:
df_web2['cancel_acco'] = df_web2['sum_new_acco'] - (df_web2['eop_sum_acco']-df_web2['bop_sum_acco'])
df_web2['cancel_rate'] = df_web2['cancel_acco']/df_web2['bop_sum_acco']
df_web2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco,cancel_acco,cancel_rate
0,2019/03/31,158235,5667,163902,NaN,NaN,NaN
1,2019/04/30,161036,6059,167095,163902.0,2866.0,0.017486
2,2019/05/31,164514,7221,171735,167095.0,2581.0,0.015446
3,2019/06/30,167583,7146,174729,171735.0,4152.0,0.024177
4,2019/07/31,171889,8815,180704,174729.0,2840.0,0.016254


In [39]:
#CSV出力
df_web2.to_csv(data_dir /f'{result}.csv', header=True ,index=None, encoding='shift=jis')

②有効口座数全体データの読込＆集計

In [61]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_cust_all}.csv",encoding='shift-jis')
df_cust_exist = pd.read_csv(data_dir /f"{input_cust_all}.csv", encoding='shift-jis')
#文字化けするので修正
df_cust_exist['age'] = df_cust_exist['age'].str.replace('～','-')

df_cust_new = pd.read_csv(data_dir /f"{input_cust}.csv",encoding='shift-jis')
df_cust_new['age'] = df_cust_new['age'].str.replace('〜','-')

In [58]:
df_cust_exist

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
3649,2008,2020,8,外商,郵送,70-74歳,男性,14913
3650,2008,2020,8,外商,郵送,75-79歳,女性,4700
3651,2008,2020,8,外商,郵送,75-79歳,男性,10120
3652,2008,2020,8,外商,郵送,80歳以上,女性,9776


In [62]:
df_cust_new

,cp,year,month,card,channel,age,gender,no_of_account
0,2009,2020,9,ゴールド,オンライン,25-29歳,女性,19
1,2009,2020,9,ゴールド,オンライン,25-29歳,男性,23
2,2009,2020,9,ゴールド,オンライン,30-34歳,女性,47
3,2009,2020,9,ゴールド,オンライン,30-34歳,男性,70
4,2009,2020,9,ゴールド,オンライン,35-39歳,女性,61
...,...,...,...,...,...,...,...,...
196,2009,2020,9,外商,郵送,70-74歳,男性,14732
197,2009,2020,9,外商,郵送,75-79歳,女性,4639
198,2009,2020,9,外商,郵送,75-79歳,男性,9878
199,2009,2020,9,外商,郵送,80歳以上,女性,9660


In [63]:
df_cust = pd.concat([df_cust_exist,df_cust_new])
df_cust

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
196,2009,2020,9,外商,郵送,70-74歳,男性,14732
197,2009,2020,9,外商,郵送,75-79歳,女性,4639
198,2009,2020,9,外商,郵送,75-79歳,男性,9878
199,2009,2020,9,外商,郵送,80歳以上,女性,9660


In [64]:
df_cust = df_cust[df_cust['cp']>= 1803] #データが2019年3月以降しか現状無し
df_cust

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
196,2009,2020,9,外商,郵送,70-74歳,男性,14732
197,2009,2020,9,外商,郵送,75-79歳,女性,4639
198,2009,2020,9,外商,郵送,75-79歳,男性,9878
199,2009,2020,9,外商,郵送,80歳以上,女性,9660


In [65]:
df_cust.to_csv(data_dir /f'{output2}.csv', header=True ,index=None, encoding='shift-jis')

In [66]:
print(df_cust.shape)
print(df_cust.index)
print(df_cust.columns)
print(df_cust.dtypes)

(3855, 8)
Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            191, 192, 193, 194, 195, 196, 197, 198, 199, 200],
           dtype='int64', length=3855)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'no_of_account'],
      dtype='object')
cp                int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
no_of_account     int64
dtype: object


In [67]:
#日付データの変換
df_cust['cp'] = df_cust['cp'] + 200000
df_cust['cp_date'] = pd.to_datetime(df_cust['cp'], format="%Y%m") + MonthEnd(1)
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019-03-31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019-03-31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019-03-31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019-03-31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019-03-31


#日付データの変換
df_cust['cp_date'] = (df_cust['cp'] + 200000)*100 + 1
df_cust['cp_date']=pd.to_datetime(df_cust['cp_date'], format='%Y%m%d')
df_cust.head()

In [68]:
#日付から時間を削除
df_cust['cp_date'] = df_cust['cp_date'].dt.strftime('%Y/%m/%d')
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019/03/31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019/03/31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019/03/31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019/03/31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019/03/31


チャネル別口座数

In [69]:
print(df['channel'].unique())

['オンライン' '店頭タブレット' '郵送']


In [70]:
q = """
        SELECT
        cp_date
        ,channel
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,channel
        """

df_cha = sqldf(q, locals())
df_cha.head()

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2019/03/31,オンライン,5747,188,5935
1,2019/03/31,店頭タブレット,95429,3145,98574
2,2019/03/31,郵送,57059,2334,59393
3,2019/04/30,オンライン,5855,205,6060
4,2019/04/30,店頭タブレット,97208,3139,100347


In [71]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,channel
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,channel
        """

df_cst_cha = sqldf(q, locals())
df_cst_cha.head()

,cp_date,channel,total_cust
0,2019/03/31,オンライン,11114
1,2019/03/31,店頭タブレット,430268
2,2019/03/31,郵送,1156364
3,2019/04/30,オンライン,11171
4,2019/04/30,店頭タブレット,430348


In [72]:
df_cha2 = pd.merge(df_cha,df_cst_cha, on =['cp_date','channel'], how = 'left')
df_cha2['regist_rate'] = df_cha2['eop_sum_acco']/df_cha2['total_cust']
df_cha2

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2019/03/31,オンライン,5747,188,5935,11114,0.534011
1,2019/03/31,店頭タブレット,95429,3145,98574,430268,0.229099
2,2019/03/31,郵送,57059,2334,59393,1156364,0.051362
3,2019/04/30,オンライン,5855,205,6060,11171,0.542476
4,2019/04/30,店頭タブレット,97208,3139,100347,430348,0.233176
5,2019/04/30,郵送,57973,2715,60688,1151197,0.052717
6,2019/05/31,オンライン,5998,235,6233,11270,0.553061
7,2019/05/31,店頭タブレット,99123,3985,103108,433097,0.238071
8,2019/05/31,郵送,59393,3001,62394,1146704,0.054412
9,2019/06/30,オンライン,6028,181,6209,11071,0.560835


In [73]:
#CSV出力
df_cha2.to_csv(data_dir /f'{result_cha}.csv', header=True ,index=None, encoding='shift=jis')

カード種類別新規登録数

In [74]:
print(df['card'].unique())

['ゴールド' 'ハウスゴールド' '一般' '外商']


In [75]:
q = """
        SELECT
        cp_date
        ,card
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,card
        """

df_card = sqldf(q, locals())
df_card.head()

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2019/03/31,ゴールド,8485.0,254,8739
1,2019/03/31,ハウスゴールド,NaN,873,873
2,2019/03/31,一般,145890.0,4149,150039
3,2019/03/31,外商,3860.0,391,4251
4,2019/04/30,ゴールド,8641.0,303,8944


In [76]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,card
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,card
        """

df_cst_card = sqldf(q, locals())
df_cst_card.head()

,cp_date,card,total_cust
0,2019/03/31,ゴールド,42171
1,2019/03/31,ハウスゴールド,38995
2,2019/03/31,一般,1361663
3,2019/03/31,外商,154917
4,2019/04/30,ゴールド,42317


In [77]:
df_card2 = pd.merge(df_card,df_cst_card, on =['cp_date','card'], how = 'left')
df_card2['regist_rate'] = df_card2['eop_sum_acco']/df_card2['total_cust']
df_card2

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2019/03/31,ゴールド,8485.0,254,8739,42171,0.207228
1,2019/03/31,ハウスゴールド,NaN,873,873,38995,0.022387
2,2019/03/31,一般,145890.0,4149,150039,1361663,0.110188
3,2019/03/31,外商,3860.0,391,4251,154917,0.027441
4,2019/04/30,ゴールド,8641.0,303,8944,42317,0.211357
...,...,...,...,...,...,...,...
71,2020/08/31,外商,25059.0,3626,28685,166199,0.172594
72,2020/09/30,ゴールド,15113.0,1118,16231,41975,0.386683
73,2020/09/30,ハウスゴールド,NaN,3555,3555,35372,0.100503
74,2020/09/30,一般,263384.0,20281,283665,1257727,0.225538


In [78]:
#CSV出力
df_card2.to_csv(data_dir /f'{result_card}.csv', header=True ,index=None, encoding='shift=jis')

年令別

In [79]:
print(df['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '75-79歳' '80歳以上' '19歳以下（20歳未満）' '20-24歳']


In [80]:
q = """
        SELECT
        cp_date
        ,age
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,age
        """

df_age = sqldf(q, locals())
df_age.head()

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2019/03/31,19歳以下（20歳未満）,245,28,273
1,2019/03/31,20-24歳,12423,446,12869
2,2019/03/31,25-29歳,20700,595,21295
3,2019/03/31,30-34歳,19202,542,19744
4,2019/03/31,35-39歳,16914,517,17431


In [81]:
print(df_cust['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '75-79歳' '80歳以上' '19歳以下（20歳未満）' '20-24歳']


In [82]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,age
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,age
        """

df_cst_age = sqldf(q, locals())
df_cst_age.head()

,cp_date,age,total_cust
0,2019/03/31,19歳以下（20歳未満）,478
1,2019/03/31,20-24歳,23281
2,2019/03/31,25-29歳,52266
3,2019/03/31,30-34歳,68757
4,2019/03/31,35-39歳,84433


In [83]:
df_age2 = pd.merge(df_age,df_cst_age, on =['cp_date','age'], how = 'left')
df_age2['regist_rate'] = df_age2['eop_sum_acco']/df_age2['total_cust']
df_age2

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2019/03/31,19歳以下（20歳未満）,245,28,273,478,0.571130
1,2019/03/31,20-24歳,12423,446,12869,23281,0.552768
2,2019/03/31,25-29歳,20700,595,21295,52266,0.407435
3,2019/03/31,30-34歳,19202,542,19744,68757,0.287156
4,2019/03/31,35-39歳,16914,517,17431,84433,0.206448
...,...,...,...,...,...,...,...
261,2020/09/30,60-64歳,28437,3968,32405,160044,0.202476
262,2020/09/30,65-69歳,18074,3022,21096,159861,0.131965
263,2020/09/30,70-74歳,11508,2404,13912,173561,0.080156
264,2020/09/30,75-79歳,3202,623,3825,119233,0.032080


In [84]:
#CSV出力
df_age2.to_csv(data_dir /f'{result_age}.csv', header=True ,index=None, encoding='utf-8-sig')